In [17]:
from keras.models import load_model
import sounddevice as sd
from scipy.io.wavfile import write
import tensorflow as tf 
import numpy as np

fs = 16000
seconds = 1

commands = np.array(['down' ,'left','right', 'up'])

def get_spectrogram(waveform):
    spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

model = load_model('my_model.h5')

In [18]:
import pygame
import random

from pygame.locals import (
    RLEACCEL,
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    K_SPACE,
    KEYDOWN,
    QUIT,
)

ppeg = 'rocket_.jpg'
epeg = 'asteroid.png'
count = lives = lc =  0
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 700

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.image.load(ppeg).convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.rect = self.surf.get_rect()
    def update(self, pressed_keys):
        if pressed_keys=='up':
            self.rect.move_ip(0, -90)
            move_up_sound.play()
        elif pressed_keys=='down':
            self.rect.move_ip(0, 90)
            move_up_sound.play()
        elif pressed_keys=='left':
            self.rect.move_ip(-90, 0)
            move_up_sound.play()
        elif pressed_keys=='right':
            self.rect.move_ip(90, 0)
            move_up_sound.play()
        
        if self.rect.left < 0:
            self.rect.left = 0
        elif self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        elif self.rect.top <= 0:
            self.rect.top = 0
        elif self.rect.bottom >= SCREEN_HEIGHT:
            self.rect.bottom = SCREEN_HEIGHT

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super(Enemy, self).__init__()
        self.surf = pygame.image.load(epeg).convert()
        self.surf.set_colorkey((255, 255, 255), RLEACCEL)
        self.rect = self.surf.get_rect(
            center=(
                random.randint(SCREEN_WIDTH + 20, SCREEN_WIDTH + 100),
                random.randint(0, SCREEN_HEIGHT),
            )
        )
        self.speed = random.randint(50, 100)
    def update(self):
        self.rect.move_ip(-self.speed, 0)
        if self.rect.right < 0:
            self.kill()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [20]:
pygame.mixer.init()
pause = False
running = True
collision_sound = pygame.mixer.Sound("explosion.mp3")
move_up_sound = pygame.mixer.Sound("move.wav")
go = pygame.mixer.Sound("game_over.wav")
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
exp = pygame.image.load('explosion03.png').convert()
ADDENEMY = pygame.USEREVENT + 2
pygame.time.set_timer(ADDENEMY, 750)
player = Player()
enemies = pygame.sprite.Group()
all_sprites = pygame.sprite.Group()
all_sprites.add(player)
clock = pygame.time.Clock()
while running:
    for event in pygame.event.get():
        if event.type == KEYDOWN:
            if event.key == K_ESCAPE:
                running = False
#             if event.key == K_SPACE:
#                 pause = not pause
        elif event.type == QUIT:
            running = False
        elif event.type == ADDENEMY:
            new_enemy = Enemy()
            enemies.add(new_enemy)
            all_sprites.add(new_enemy)
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
    sd.wait()  # Wait until recording is finished
    write('output1.wav', fs, myrecording)  # Save as WAV file
    x = tf.squeeze(myrecording, axis=-1)
    waveform = x
    x = get_spectrogram(x)
    x = x[tf.newaxis,...]
    prediction = model(x)
    predictions = tf.nn.softmax(prediction[0])
    if max(np.array(predictions))<=.35:
        pressed_keys = 'continue'
    else:
        direction = commands[max(np.array(predictions))==np.array(predictions)]
        pressed_keys = direction[0]
    player.update(pressed_keys)
    enemies.update()
    screen.fill((135, 206, 235))
    screen.blit(player.surf, player.rect)
    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)
    if pygame.sprite.spritecollideany(player, enemies):
        screen.blit(exp, player.rect)
        pygame.display.flip()
        collision_sound.play()
        pygame.time.wait(3500)
        go.play()
        pygame.time.wait(900)
        player.kill()
        running = False
    clock.tick(100)
    pygame.display.flip()
    count+=1
            
pygame.quit()
print('Score :', count)

Score : 39 0 0
